In [1]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath('../scripts'))
from preprocessing import *

In [2]:
# data pre processing
train = pd.read_csv('../data/train.csv', low_memory=False)
preprocessing_scaler(train)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,IsWeekend,IsHoliday,DaysToHoliday,DaysAfterHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1,False,0,NaN,5.0
1,2,5,2015-07-31,6064,625,1,1,0,1,False,0,NaN,5.0
2,3,5,2015-07-31,8314,821,1,1,0,1,False,0,NaN,5.0
3,4,5,2015-07-31,13995,1498,1,1,0,1,False,0,NaN,5.0
4,5,5,2015-07-31,4822,559,1,1,0,1,False,0,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,False,1,4.0,NaN
1017205,1112,2,2013-01-01,0,0,0,0,a,1,False,1,4.0,NaN
1017206,1113,2,2013-01-01,0,0,0,0,a,1,False,1,4.0,NaN
1017207,1114,2,2013-01-01,0,0,0,0,a,1,False,1,4.0,NaN


In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline

In [6]:
test = pd.read_csv('../data/test.csv')

# Convert categorical columns to numeric using LabelEncoder or OneHotEncoder
label_encoder = LabelEncoder()

# Example: Encode categorical columns
categorical_columns = ['StoreType', 'Assortment', 'StateHoliday']
for col in categorical_columns:
    train[col] = label_encoder.fit_transform(train[col].astype(str))
    test[col] = label_encoder.transform(test[col].astype(str))

# Prepare the training data
X_train = train.drop(['Sales', 'Date'], axis=1)  # Drop 'Sales' and 'Date'
y_train = train['Sales']

# Prepare the test data
X_test = test.drop(['Date'], axis=1)

# Create and fit the pipeline
model_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scale the data
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))  # Random Forest Regressor
])

# Fit the model on the training data
model_pipeline.fit(X_train, y_train)

# Make predictions on the test data
test_predictions = model_pipeline.predict(X_test)

# Save the predictions to a file
test['PredictedSales'] = test_predictions
test[['Store', 'Date', 'PredictedSales']].to_csv('test_predictions.csv', index=False)

KeyError: 'StoreType'

In [7]:
print(train.columns)
print(test.columns)


Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'IsWeekend', 'IsHoliday',
       'DaysToHoliday', 'DaysAfterHoliday'],
      dtype='object')
Index(['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday'],
      dtype='object')
